In [ ]:
"""Imports Required"""

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from scipy.stats import norm, skew
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.family'] = 'serif'

In [38]:
"""Model Parameters"""

S0 = 100
K = S0
mu = 0.1
sigma = 0.2
r = 0.03
T_put = 0.5
T_call = 0.75
T = 0.5
N = int(252 * T)
n_paths = 10000
shares_per_contract = 100
equity_transaction = 0.005
option_transaction = 0.01

In [3]:
def simulate_gbm(S0, mu, sigma, T, N, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)
    dt = T / N
    paths = np.zeros((n_paths, N + 1))
    paths[:, 0] = S0
    dW = np.random.normal(0, np.sqrt(dt), size=(n_paths, N))
    for t in range(1, N + 1):
        paths[:, t] = paths[:, t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * dW[:, t - 1])
    return paths

def create_base_df(path, K, r, sigma, T_put, T_call):
    N = len(path) - 1
    step = np.arange(N + 1)
    dt = 1 / 252
    T_put_vec = np.maximum(T_put - step * dt, 0)
    T_call_vec = np.maximum(T_call - step * dt, 0)
    put_price = bs_put(path, K, T_put_vec, r, sigma)
    put_delta = bs_put_delta(path, K, T_put_vec, r, sigma)
    put_gamma = bs_put_gamma(path, K, T_put_vec, r, sigma)
    call_price = bs_call(path, K, T_call_vec, r, sigma)
    call_delta = bs_call_delta(path, K, T_call_vec, r, sigma)
    call_gamma = bs_call_gamma(path, K, T_call_vec, r, sigma)

    return pd.DataFrame({
        'spot': path,
        'put_dte': (T_put_vec * 252).astype(int),
        'call_dte': (T_call_vec * 252).astype(int),
        'put_price': put_price,
        'put_delta': put_delta,
        'put_gamma': put_gamma,
        'call_price': call_price,
        'call_delta': call_delta,
        'call_gamma': call_gamma,
    })


def bs_put(S, K, T_put, r, sigma):
    S = np.maximum(S, 1e-8)
    T_put = np.maximum(T_put, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T_put) / (sigma * np.sqrt(T_put))
    d2 = d1 - sigma * np.sqrt(T_put)
    return K * np.exp(-r * T_put) * norm.cdf(-d2) - S * norm.cdf(-d1)

def bs_put_delta(S, K, T_put, r, sigma):
    S = np.maximum(S, 1e-8)
    T_put = np.maximum(T_put, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T_put) / (sigma * np.sqrt(T_put))
    return norm.cdf(d1) - 1

def bs_put_gamma(S, K, T_put, r, sigma):
    S = np.maximum(S, 1e-8)
    T_put = np.maximum(T_put, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T_put) / (sigma * np.sqrt(T_put))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T_put))

def bs_call(S, K, T_call, r, sigma):
    S = np.maximum(S, 1e-8)
    T_call = np.maximum(T_call, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T_call) / (sigma * np.sqrt(T_call))
    d2 = d1 - sigma * np.sqrt(T_call)
    return S * norm.cdf(d1) - K * np.exp(-r * T_call) * norm.cdf(d2)

def bs_call_delta(S, K, T_call, r, sigma):
    S = np.maximum(S, 1e-8)
    T_call = np.maximum(T_call, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T_call) / (sigma * np.sqrt(T_call))
    return norm.cdf(d1)

def bs_call_gamma(S, K, T_call, r, sigma):
    S = np.maximum(S, 1e-8)
    T_call = np.maximum(T_call, 1e-8)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T_call) / (sigma * np.sqrt(T_call))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T_call))

def calc_gamma_ratio(df):
    df['gamma_ratio'] = df['put_gamma']/df['call_gamma']
    return df

def calc_calls_held(df):
    df['calls_held_change'] = df['gamma_ratio'].diff()
    df.loc[0, 'calls_held_change'] = df.loc[0, 'gamma_ratio']
    return df

def calc_delta_port(df):
    df['net_port_delta'] = (df['put_delta'] * -1) + (df['gamma_ratio'] * df['call_delta'])
    return df

def calc_shares_held(df):
    df['shares_held'] = -df['net_port_delta'] * shares_per_contract
    return df

def calc_portfolio_cost(df):
    df['rebalancing_cost'] = ((df['shares_held'] -df['shares_held'].shift(1)) * df['spot']) + (abs((df['shares_held'] - df['shares_held'].shift(1)) * df['spot']) * equity_transaction) + ((df['gamma_ratio'] - df['gamma_ratio'].shift(1)) * (df['call_price'] * shares_per_contract)) + ((df['gamma_ratio'] - df['gamma_ratio'].shift(1)) * (df['call_price'] * shares_per_contract) * option_transaction)
    df.loc[0, 'rebalancing_cost'] = (df['shares_held'].iloc[0] * df['spot'].iloc[0]) + (abs(df['shares_held'].iloc[0] * df['spot'].iloc[0]) * equity_transaction) - (shares_per_contract * df['put_price'].iloc[0]) + (df['put_price'].iloc[0] * shares_per_contract * option_transaction) + (shares_per_contract * (df['call_price'].iloc[0] * df['gamma_ratio'].iloc[0])) + (df['call_price'].iloc[0] * df['gamma_ratio'].iloc[0] * shares_per_contract * option_transaction)
    df['portfolio_cost'] = df['rebalancing_cost'].cumsum()
    return df

def calc_call_pnl(df):
    df['call_pnl'] = df['gamma_ratio'].shift(1) * (df['call_price'] - df['call_price'].shift(1))
    df['call_pnl'] = df['call_pnl'].fillna(0) * shares_per_contract
    return df

def calc_put_pnl(df):
    df['put_pnl'] = df['put_price'].shift(1) - df['put_price']
    df['put_pnl'] = df['put_pnl'].fillna(0) * shares_per_contract
    return df

def calc_shares_pnl(df):
    df['shares_pnl'] = df['shares_held'].shift(1) * (df['spot'] - df['spot'].shift(1))
    df.loc[0, 'shares_pnl'] = 0
    return df

def add_interest(df, r):
    df['interest'] = df['portfolio_cost'].shift(1) * (np.exp(-r / 252) - 1)
    df.loc[0, 'interest'] = 0
    return df

def calc_overnight_pnl(df):
    df['overnight_pnl'] = df['call_pnl'] + df['put_pnl'] + df['shares_pnl'] + df['interest']
    return df

In [ ]:
"""Run the simulation to create a dataframe for each path"""

paths = simulate_gbm(S0, mu, sigma, T, N, n_paths)

dfs = []
for i in range(len(paths)):
    df = create_base_df(paths[i], K, r, sigma, T_put, T_call)                      
    df = calc_gamma_ratio(df)
    df = calc_calls_held(df)
    df = calc_delta_port(df)
    df = calc_shares_held(df)
    df = calc_call_pnl(df)
    df = calc_put_pnl(df)
    df = calc_shares_pnl(df)
    df = calc_portfolio_cost(df)
    df = add_interest(df, r)
    df = calc_overnight_pnl(df)
    
    dfs.append(df)

In [ ]:
"""Inspect a specfic path by accessing dataframe [n]"""

dfs[0]

In [ ]:
"""Print the analysis results"""

overnight_totals = [df['overnight_pnl'].sum() for df in dfs]
overnight_pnl_per_path = np.array(overnight_totals)
mean = np.mean(overnight_pnl_per_path)
std_dev = np.std(overnight_pnl_per_path)
alpha = 0.05
var = np.percentile(overnight_pnl_per_path, 100 * alpha)
cvar = overnight_pnl_per_path[overnight_pnl_per_path <= var].mean()
skewness = skew(overnight_pnl_per_path)
eqr = np.percentile(overnight_pnl_per_path, 99) - np.percentile(overnight_pnl_per_path, 1)

print("Mean:", mean)
print("Standard Deviation:", std_dev)
print(f"VaR at {alpha*100}%:", var)
print(f"CVaR at {alpha*100}%:", cvar)
print("Skewness:", skewness)
print("EQR:", eqr)

In [ ]:
"""Plot the Overnight PnL Distributions"""

sns.histplot(overnight_pnl_per_path, bins=1000, color='skyblue')
plt.title("Delta-Gamma Hedge Overnight PnL Distribution")
plt.xlabel("PnL")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
"""Plot the path simulation"""

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_title("Delta-Gamma Hedge Simulation")
ax1.set_xlabel("Step")
ax1.plot(df.index, df['spot'], label='Spot Price', color='black')
ax1.plot(df.index, df['put_price'], label='Put Price', color='purple')
ax1.plot(df.index, df['call_price'], label='Call Price', color='skyblue')
ax1.plot(df.index, df['overnight_pnl'], label='Overnight PnL', color='red')
ax1.plot(df.index, df['shares_held'], label='Shares Held', color='blue')
ax1.tick_params(axis='y')
ax1.legend(loc='upper right')

plt.tight_layout()
plt.show()